In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [4]:
# Create Spark Session.
spark = SparkSession.builder.appName("task2").getOrCreate()

24/11/12 13:25:37 WARN Utils: Your hostname, facu-ASUS-TUF-Gaming-F15-FX507ZC4-FX507ZC4 resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlo1)
24/11/12 13:25:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 13:25:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
import os
parent_path = os.path.dirname(os.getcwd())
os.path.join(parent_path, 'data-engineering-test/')

'/home/facu/Documents/IFCO/data-engineering-test'

In [37]:
df = spark.read.csv("/home/facu/Documents/IFCO/data-engineering-test/resources/orders.csv" ,header=True,sep=";", inferSchema=True)

In [38]:
df_cleaned =  (df
            .withColumn("cleaned_json",F.regexp_replace(F.col("contact_data"),"^['\"]|['\"]$",""))
            .withColumn("cleaned_json", F.regexp_replace(F.col("cleaned_json"),'""','"'))
)

In [61]:
df_cleaned.select("cleaned_json").collect()

[Row(cleaned_json='[{ "contact_name":"Curtis", "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]'),
 Row(cleaned_json='[{ "contact_name":"Maria", "contact_surname":"Theresa", "city":"Calcutta"}]'),
 Row(cleaned_json='[{ "contact_name":"Para", "contact_surname":"Cetamol", "city":"Frankfurt am Oder", "cp": 3934}]'),
 Row(cleaned_json=None),
 Row(cleaned_json=None),
 Row(cleaned_json='[{ "contact_name":"John", "contact_surname":"Krasinski", "city":"New York", "cp": "1203"}]'),
 Row(cleaned_json=None),
 Row(cleaned_json='[{ "contact_name":"Jennifer", "contact_surname":"Lopez", "city":"Esplugues de Llobregat"}]'),
 Row(cleaned_json='{ "contact_name":"Liav", "contact_surname": "Ichenbaum", "city":"Tel Aviv"}'),
 Row(cleaned_json='[{ "contact_name":"Curtis", "contact_surname":"Jackson", "city":"Chicago", "cp": "12345"}]'),
 Row(cleaned_json='[{ "contact_name":"Anthony", "contact_surname":"Pap", "city":"Barcelona", "cp": 8023}]'),
 Row(cleaned_json='[{ "contact_name":"Natalia", "c

In [62]:
schema = T.ArrayType(T.StructType([
    T.StructField("contact_name", T.StringType(), True),
    T.StructField("contact_surname", T.StringType(), True),
    T.StructField("city", T.StringType(), True),
    T.StructField("cp", T.StringType(), True)
]))

df_parsed = df_cleaned.withColumn(
        "parsed_json",
        F.from_json(F.col("cleaned_json"), schema)
    )

In [63]:
df_exploded = df_parsed.withColumn(
        "exploded_json",
        F.explode_outer(F.col("parsed_json"))
    )

In [71]:
df_final = df_exploded.select(
        "order_id",
        "date",
        "company_id",
        "company_name",
        "crate_type",
        F.concat(
            F.coalesce(F.col("exploded_json.contact_name"), F.lit("John")),
            F.lit(" "),
            F.coalesce(F.col("exploded_json.contact_surname"), F.lit("Doe"))
        ).alias("contact_full_name"),
        
        F.concat(
            F.coalesce(F.col("exploded_json.city"), F.lit("Unknown")),
            F.lit(", "),
            F.coalesce(F.col("exploded_json.cp"), F.lit("UNK00")).cast("string")
        ).alias("contact_address"),
        "salesowners"
    )

In [70]:
df_exploded.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- company_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- crate_type: string (nullable = true)
 |-- contact_data: string (nullable = true)
 |-- salesowners: string (nullable = true)
 |-- cleaned_json: string (nullable = true)
 |-- parsed_json: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- contact_name: string (nullable = true)
 |    |    |-- contact_surname: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- cp: string (nullable = true)
 |-- exploded_json: struct (nullable = true)
 |    |-- contact_name: string (nullable = true)
 |    |-- contact_surname: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- cp: string (nullable = true)



In [66]:
df_final.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- company_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- crate_type: string (nullable = true)
 |-- contact_name: string (nullable = false)
 |-- contact_surname: string (nullable = false)
 |-- city: string (nullable = false)
 |-- cp: string (nullable = false)

